In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Gitter
n = 10
x0 = np.linspace(0, 1, n)
x1 = np.linspace(0, 1, n)
x2 = np.linspace(0, 1, n)
x3 = np.linspace(0, 1, n)

X0, X1, X2, X3 = np.meshgrid(x0, x1, x2, x3, indexing='ij')
points4d = np.stack([X0.ravel(), X1.ravel(), X2.ravel(), X3.ravel()], axis=1)
u_vals = points4d.sum(axis=1)

# Werte vorbereiten pro Zeitschnitt (x3 fix)
frames = []
t_vals = np.unique(points4d[:, 3])
for t in t_vals:
    mask = np.isclose(points4d[:, 3], t)
    pts = points4d[mask][:, :3]  # nur x0, x1, x2
    u = u_vals[mask]
    frames.append((pts, u, t))

# 🎥 Matplotlib 3D-Animation

fig = plt.figure(figsize=(6,5))
ax = fig.add_subplot(projection='3d')

# Initialer Dummy-Scatter für das Colorbar-Setup
sc = ax.scatter([], [], [], c=[], cmap="viridis", vmin=0, vmax=4)

# Hier wird die Colorbar erstellt – das ist neu:
cbar = fig.colorbar(sc, ax=ax, shrink=0.75, pad=0.1)
cbar.set_label("u(x)", rotation=270, labelpad=15)

sc = ax.scatter([], [], [], c=[], cmap="viridis", vmin=0, vmax=4)

def init():
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_zlim(0, 1)
    ax.set_title("u(x) = x₀ + x₁ + x₂ + x₃")
    return sc,

def update(frame):
    pts, u, t = frame
    sc._offsets3d = (pts[:,0], pts[:,1], pts[:,2])
    sc.set_array(u)
    ax.set_title(f"x₃ = {t:.2f}")
    return sc,

ani = FuncAnimation(fig, update, frames=frames, init_func=init, blit=False, repeat=True)

plt.close()
from IPython.display import HTML
HTML(ani.to_jshtml())


In [ ]:
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# Gitter
n = 20
x0 = np.linspace(0, 1, n)
x1 = np.linspace(0, 1, n)
x2 = np.linspace(0, 1, n)
x3 = np.linspace(0, 1, n)

X0, X1, X2, X3 = np.meshgrid(x0, x1, x2, x3, indexing='ij')
U = X0 + X1 + X2 + X3

# Interaktive Zeit-Slice mit Plotly
def plot_slice(t_index):
    u_slice = U[:, :, :, t_index]
    fig = go.Figure(data=go.Volume(
        x=X0[:,:,:,t_index].flatten(),
        y=X1[:,:,:,t_index].flatten(),
        z=X2[:,:,:,t_index].flatten(),
        value=u_slice.flatten(),
        isomin=0,
        isomax=4,
        opacity=0.1, # für Transparenz
        surface_count=20,
        colorscale='Turbo'
    ))
    fig.update_layout(scene=dict(
        xaxis_title='x₀',
        yaxis_title='x₁',
        zaxis_title='x₂'
    ), title=f"x₃ = {x3[t_index]:.2f}")
    fig.show()

# Slider
slider = widgets.IntSlider(value=0, min=0, max=n-1, step=1, description='t-index')
widgets.interact(plot_slice, t_index=slider)


interactive(children=(IntSlider(value=0, description='t-index', max=19), Output()), _dom_classes=('widget-inte…

<function __main__.plot_slice(t_index)>